# Adding days per month from date range to a dataframe

Suppose you have a dataset with a column of start dates and column of end dates.  For example:

In [2]:
import pandas as pd
import calendar

date_df = pd.DataFrame({
    "START_TM": ['2/15/2010', '2/15/2010', '3/16/2010'],
    "END_TM": ['4/18/2010', '2/18/2010', '5/20/2010']
})
date_df["START_TM"] = date_df["START_TM"].astype('datetime64')
date_df["END_TM"] = date_df["END_TM"].astype('datetime64')
date_df

,START_TM,END_TM
0,2010-02-15,2010-04-18
1,2010-02-15,2010-02-18
2,2010-03-16,2010-05-20


Our goal is to count the number of days in each month this range of dates falls over.

We start by adding columns for each month:

In [4]:
months = {calendar.month_name[i]:[0 for _ in range(date_df.shape[0])] for i in range(1, 13)}
for m in months:
    date_df[m] = [0 for _ in range(date_df.shape[0])]

In [5]:
date_df

,START_TM,END_TM,January,February,March,April,May,June,July,August,September,October,November,December
0,2010-02-15,2010-04-18,0,0,0,0,0,0,0,0,0,0,0,0
1,2010-02-15,2010-02-18,0,0,0,0,0,0,0,0,0,0,0,0
2,2010-03-16,2010-05-20,0,0,0,0,0,0,0,0,0,0,0,0


We'll use these two functions:

In [6]:
def insert_days_per_month(outer_row):
    dpm = days_per_month(outer_row)
    for index, inner_row in dpm.iterrows():
        outer_row[inner_row['Month']] = inner_row['NumDays']
    return(outer_row)

def days_per_month(row):
    # Remove the [1: ] at the end of this line if your count is off by one
    s = pd.Series(index = pd.date_range(row[0], row[1]))[1: ] 
    days_in_month = s.resample('MS').size().to_period('m').\
    rename_axis('Month').reset_index(name = 'NumDays')
    days_in_month['Month'] = days_in_month['Month'].apply(
        lambda x: calendar.month_name[x.month])
    return(days_in_month)

We can get the desired result with apply:

In [11]:
date_df = date_df.apply(lambda x: insert_days_per_month(x), axis = 1)

In [12]:
date_df

,START_TM,END_TM,January,February,March,April,May,June,July,August,September,October,November,December
0,2010-02-15,2010-04-18,0,13,31,18,0,0,0,0,0,0,0,0
1,2010-02-15,2010-02-18,0,3,0,0,0,0,0,0,0,0,0,0
2,2010-03-16,2010-05-20,0,0,15,30,20,0,0,0,0,0,0,0
